# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
file_to_load = "../output_data/cities.csv"
# Read Purchasing File and store into Pandas data frame
cities_data_1 = pd.read_csv(file_to_load)
cities_data_1.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mataura,-46.1927,168.8643,53.47,80,100,0.76,NZ,1643904013
1,1,Geraldton,-28.7667,114.6000,72.01,60,0,4.61,AU,1643904015
2,2,Berlevåg,70.8578,29.0864,22.80,79,75,5.75,NO,1643904017
3,3,‘Āhuimanu,21.4447,-157.8378,71.85,81,15,1.01,US,1643904019
4,4,Smidovich,48.6000,133.8167,-23.71,100,46,2.86,RU,1643904020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [27]:
locations = cities_data_1[["Lat", "Lng"]]
humidity = cities_data_1['Humidity'].astype(int)
maxhumidity = humidity.max()

# # Plot Heatmap
figure = gmaps.figure()
# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=2)
# # Add layer
figure.add_layer(heat_layer)
# # Display figure
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [78]:
print(f'---Please enter a value between 70 and 95 (no decimals)---\n')
while True:
    try:
        min_t = int(input('What is the lowest temperature you desired?'))
    except ValueError:
        print(f'{bcolors.WARNING}Value must be a number.{bcolors.RESET}')
    else:
        if 69 < min_t < 96:
            break
        else:
            print(f'{bcolors.FAIL}Value must be between 70 and 95!{bcolors.RESET}')
while True:
    try:
        max_t = int(input('What about the hotest?'))
    except ValueError:
        print(f'{bcolors.WARNING}Value must be a number.{bcolors.RESET}')
    else:
        if min_t+1 < max_t < 96:
            break
        else:
            print(f'{bcolors.FAIL}Value must be between {min_t+2} and 95{bcolors.RESET}')
print(f'---Please enter a value between 0.0 and 8.9---' )
while True:
    try:
        wind_s = float(input('How windy do you like your vacation?'))
    except ValueError:
        print(f'{bcolors.WARNING}Value must be a number.{bcolors.RESET}')
    else:
        if 0.0 <= wind_s <= 8.9:
            break
        else:
            print(f'{bcolors.FAIL}Value must be between 0.0 and 8.9!{bcolors.RESET}')

---Please enter a value between 70 and 95 (no decimals)---

What is the lowest temperature you desired?70
What about the hotest?g
Value must be a number.
What about the hotest?50
Value must be between 72 and 95
What about the hotest?80
---Please enter a value between 0.0 and 8.0---
How windy do you like your vacation?f
Value must be a number.
How windy do you like your vacation?8


In [77]:
vacation_df = cities_data_1[(cities_data_1['Max Temp'].between(min_t,max_t)) & (cities_data_1['Wind Speed']<=wind_s) & (cities_data_1['Humidity']<=45)]
vacation_df.reset_index(inplace=True)
del vacation_df['index']
print(f'Threre are --{len(vacation_df)}-- cities within your criteria!')
vacation_df

Threre are --9-- cities within your criteria!


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,99,Bondo,3.8146,23.6866,91.06,23,97,1.45,CD,1643904107
1,156,Orán,-23.1322,-64.3260,92.37,33,32,2.86,AR,1643904152
2,238,Obo,5.3998,26.4869,92.39,17,46,5.64,CF,1643904218
3,255,Rawson,-43.3002,-65.1023,78.39,39,41,7.31,AR,1643904067
4,282,Gambēla,8.2500,34.5833,92.26,24,96,5.12,ET,1643904252
5,343,Kabo,7.6994,18.6290,94.64,11,23,7.25,CF,1643904300
6,395,Reconquista,-29.1500,-59.6500,93.92,32,98,7.76,AR,1643904342
7,441,Bobo-Dioulasso,11.1771,-4.2979,90.77,6,20,6.26,BF,1643904378
8,506,Nizwá,22.9333,57.5333,72.91,25,0,1.16,OM,1643904266


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
hotel_df = []
for x in range(len(vacation_df)):
    lat = vacation_df.loc[x]['Lat']
    lng = vacation_df.loc[x]['Lng']
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
     
    try:
        hotel_df.append(response['results'][0]['name'])
    except:
        hotel_df.append("")

In [39]:
vacation_df["Hotel Name"] = hotel_df
vacation_df = vacation_df.dropna(how='any')

In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]]

In [41]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
the_hotels = gmaps.figure()
the_hotels.add_layer(markers)

# Display figure
the_hotels

Figure(layout=FigureLayout(height='420px'))